In [1]:
# Import OS to interact with local computer operating system
import sys
import requests
from types import ModuleType
# Import the datetime and date classes from the datetime module, and  for working with dates.
from datetime import datetime, date
#Import markdown and display
from IPython.display import Markdown as md, display
# Last calendar day of the month and day of the week for first day
import calendar

# Ipython not builtin, install if  necessary with quiet flag (-q)
try:
    from IPython.display import Markdown, display
except:
    !pip -q install IPython
    from IPython.display import Markdown, displayr

# Pandas not builtin, install if  necessary with quiet flag (-q)
try:
    import pandas as pd
except:
    !pip -q install pandas
    import pandas as pd

# NumPy not builtin, install if  necessary with quiet flag (-q)
try:
    import numpy as np
except:
    !pip -q install numpy
    import numpy as np

# dateutilnot builtin, install if  necessary with quiet flag (-q)
try:
    from dateutil.relativedelta import relativedelta
except:
    !pip -q install python-dateutil
    from dateutil.relativedelta import relativedelta
    from dateutil.easter import easter

In [2]:
import requests, sys
from types import ModuleType
# Define the URL of the Python module to be downloaded from Dropbox.
# The 'dl=1' parameter in the URL forces a direct download of the file content.
url= 'https://www.dropbox.com/scl/fi/4y5hjxlfphh1ngvbgo77q/\
module_-basic_concepts_fixed_income.py?rlkey=6oxi7mgka42veaat79hcv8boz&st=87sztshr&dl=1'
module_name='basic_concepts_fixed_income'
# Send an HTTP GET request to the URL and store the server's response.
try:
  response=requests.get(url)
  # Raise an exception for bad status codes (like 404 Not Found)
  response.raise_for_status()
  module= ModuleType(module_name)
  #Code contained in response.text executed
  exec(response.text, module.__dict__)
  # Module added to sys
  sys.modules[module_name]=module
except requests.exceptions.RequestException as e:
    print(f"❌ Error: Could not fetch module from URL. {e}")
except Exception as e:
    print(f"❌ Error: Failed to execute or import the module. {e}")

# Now that 'basic_concepts_fixed_income' exists in the notebook, import the specific functions
from basic_concepts_fixed_income import (validate_date,
                                         scheduled_pay_dates,
                                         adjust_bond_pay_dates)

In [3]:
def bond_pay_data(maturity,coupon,settlement=None,freq=2):
  '''
  Function calculates payment Dates And Amounts.
  maturity is a datetime object and coupon is a real number.
  Required arguments are maturity and annual coupon.
  If provided, the value of settlement is a datetime object;
  otherwise defaults to date.today()
  freq defaults to semi-annual but accepts freq equal
  to 1 for annual, 2 for semi-annal, 4 for quarterly, and 12 for monthly.
  The function assumes a par value of 100.
  Returns Numpy arrays of dates and amounts.

   Raises:
      TypeError: If maturity or settlement are not datetime objects.
      ValueError: If inputs are not logically valid (e.g., negative coupon,
                  maturity before settlement).
  '''
  from datetime import datetime,date
  from dateutil.relativedelta import relativedelta
  import pandas as pd
  import numpy as np
  from IPython.display import display, Markdown as md

  #Validate the data- maturity, coupon, settlement, freq
  #maturity
  maturity=validate_date(maturity)

  #coupon
  try:
      coupon = float(coupon)
      if coupon < 0:
          raise ValueError("coupon rate cannot be negative.")
  except (ValueError, TypeError):
      raise ValueError("coupon must be a valid number.")

  #settlement
  if settlement is None:
      settlement = date.today()
  else:
      settlement=validate_date(settlement)

  #freq
  if int(freq) not in [1,2,4,12]:
      display(md(f"### ⚠️  your assigned freq {freq} it must be (1, 2, 4, or 12)\
      \n  ###    semi-annual assumed (2)."))
      freq=int(2)

  if coupon==0:
    #Adjust maturity for nonsettlement day and return date and face value
    adjust_maturity=adjust_bond_pay_dates(maturity)
    return np.array([adjust_maturity]),np.array([100.0])

  #get scheduled payment dates from helper function scheduled_pay_dates
  scheduled_dates=scheduled_pay_dates(maturity,settlement,freq)

  #NumPy array of pay_dates: list comprehension with adjust_bond_pay_dates
  pay_dates=np.array([adjust_bond_pay_dates(scheduled) for scheduled in scheduled_dates])

  #calculate payments
  #coupon divided by freq at each date
  pay=np.full(len(pay_dates),coupon/freq)

  #Add principal payment as last cash payment
  pay[-1]+=100

  return pay_dates,pay

In [4]:
maturity=date(2028,2,29)
settlement=date(2025,1,21)
coupon=4
bond_pay_data(maturity,coupon,settlement=settlement)

(array([datetime.date(2025, 2, 28), datetime.date(2025, 9, 2),
        datetime.date(2026, 3, 2), datetime.date(2026, 8, 31),
        datetime.date(2027, 3, 1), datetime.date(2027, 8, 31),
        datetime.date(2028, 2, 29)], dtype=object),
 array([  2.,   2.,   2.,   2.,   2.,   2., 102.]))